In [ ]:
%reload_ext autoreload
%autoreload 2


%matplotlib inline

# TorchServe

For installation, please refer to:  
https://github.com/pytorch/serve

### model.py

In [ ]:
!pygmentize ../deployment/model.py

### handler.py

Reference:  
- https://github.com/pytorch/serve/tree/master/ts/torch_handler

In [ ]:
!pygmentize ../deployment/handler.py

### Archive the Model

```bash
torch-model-archiver --model-name fastunet --version 1.0 --model-file deployment/model.py --serialized-file model_store/fasti_unet_weights.pth --export-path model_store --handler deployment/handler.py -f
```

### Serve the Model

```bash
torchserve --start --ncs --model-store model_store --models fastunet.mar
```

### Call API

```bash
time http POST http://127.0.0.1:8080/predictions/fastunet/ @sample/Seq05VD_f00210.png
```

### Sample Response

```bash
HTTP/1.1 200
Cache-Control: no-cache; no-store, must-revalidate, private
Expires: Thu, 01 Jan 1970 00:00:00 UTC
Pragma: no-cache
connection: keep-alive
content-length: 131101
x-request-id: 96c25cb1-99c2-459e-9165-aa5ef9e3a439

{
  "base64_prediction": "GhoaGhoaGhoaGhoaGhoaGhoaGh...ERERERERERERERERERERER"
}

real    0m0.979s
user    0m0.280s
sys     0m0.039s
```

### Decode and Visualise Returned Prediction from TorchServe

In [ ]:
response = {
    "base64_prediction": "GhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaGhoaFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaGhoVFRoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaGhoaGhUVGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaGhoVGhUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoaFRUVFRoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaGhoVFRUVGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaFRUVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUaGhoVFRUaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRoaFRUVGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVGhoaBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUEBAQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVBAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVBAQEBAQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBBUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUYGBgEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQEFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRUVFRgYGAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBAQVFRUVFRUVFRUVFRUVFRUVFRUVFQQVFRUVFRUVFRUVFRUVFRUVFRUVGBgYGAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQEBBUVFRUVFRUVFRUVFRUVFRUVBAQEBAQEBAQVFRUVFRUVFRUVFRUVFRUYGBgYBAQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBAQEBBUVFRUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEFRUVFRUVFRUVFRUVFRgYGAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQEBAQEFRUVFRUVFRUVFRUVFRUVBAQEBAQEBAQEBAQEFRUVFRUVFRUVFRUVGBgYBAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBAQEBAQVFRUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEBAQVFRUVFRUVFRUVFRUYGBgEBAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQEBAQEBBUVFRUVFRUVFRUVFQQEBAQEBAQEBAQEBAQEBBUVFRoaFRUVFRUVFRgYGAQEBAQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBAQEBAQEFRUVFRUVFRUVFRUEBAQEBAQEBAQEBAQEBAQEBBoaGhoEFRUVFRUVGBgYBAQEBAQEBAQEBAQEBAQEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGgQEBBoaGhoaGhoaGhoEBAQEBAQEBAQEFRUVFRUVFRUEBAQEBAQEBAQEBAQEBAQEBAQaGhoaGgQEBBUVFRUEGBgEBAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaBAQEBBoaGhoaGhoaGgQEBAQEBAQEBAQEFRUVFRUVBAQEBAQEBAQEBAQEBAQEBAQEGhoaGhoaGgQEBBUVFQQYBAQEBAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoaGhoaGhoEGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGgQEBAQEGhoaGhoaGhoaBAQEBAQEBAQEBAQEFRUVFQQEBAQEBAQEBAQEBAQEBAQEBAQaGhoaGhoaGgQEBBUEBAQEBAQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGhoaGgQaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoEBAQEBAQEGhoaGhoaGhoEBAQEBAQEBAQEBAQVFRUEBAQEBAQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoaGgQEBBoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGhoaGgQEBAQEBAQaGhoaGhoaGhoEBAQEBAQEBAQEBBUVBAQEBAQEBAQEBAQEBAQEBAQEBAQEGhoaGhoaGhoEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaGhoEBAQEGhoaGhoaCQkJCQkJGhoaGhoaGhoaGhoaGhoaGhoEBAQEBAQEBBoaGhoaGhoaHh4eBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQaGhoaGhoaGgQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEGhoaGhoaGhoaGgQEBAQaGhodCQkJCQkJCQkJCQkJCQkJCQkJGhoaGhoaHh4EBAQEBAQeGhoaGgkJCQkJCR4eHh4EBAQeBB4EBAQEBQUFBQQEBAQEBAQEBAQEBAQEBAkJGhoaGhoaBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQaGhoaGhoaGhoEBAQEBBoaHR0JCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCR4eHh4eHh4eHh4FBQUFBQUFBQUFBQUFBQUFBQQEBAkJCQkJCQkJCQkEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBBoaGhoaGhoaBAQEBAQEHR0JCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCR4eHh4eHh4eHgUFBQUFBQUFBQUFBQUFBQUFBQUECQkJCQkJCQkJCQkEBAQEBAQEBAQEBAQEBAQEBAQEBAQEBAQEHhoaGhoaBAQEBAQEBAQdHQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkeHh4EHh4TExMTBQUFBQUFBQUFBQUFBQUFBQUFBQUJCQkJCQkJCQkJCQQEBAQEBAQEBBoEBAQEBAQEBAQEBAQEBAQeHh4eHh4EBAQEBAQEBB0dCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkfHx8eHhMTExMTExEREQUFBQUFBQUFBQUFBQUFBQUFBR4eCQkJCQkJCQkJBAQEBAQEHh4eHh4eBAQEBAQEBAQEBAQEHh4eHh4eHh4eBAQEBAQEHR0JCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJHx8fHx8fHx8TExMTExMREREREREREREREREFBQUFBQUFBQUFBR4eHx8fHx8JCQkEBAQEBB4eCQkJCQkJCQkeBAQEHh4eHh4eHh4eHh4eHh4EBAQEBAQfHx8fCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJCQkJHx8fHx8fHx8fHx8fHx8TExMTExMTEREREREREREREREREREREREFBQUFBQUTExMTHx8fHx8fHh4EBAQeHgkJCQkJCQkJCQkJHh4eHh4eHh4eHh4eHh4eHgUFBQUFBR8fHx8fHx8fHx8fHx8JCQkJCQkJCQkfHx8fHx8fHx8fHx8fHx8fHx8fHxMTExMTExMTExMTExMRERERERERERERERERERERERERERERERETExMTExMTExMTHh4eHh4fHwkJCQkJCQkJCQkJHh4eHh4eHh4eHh4eHgUFBQUFBQUFHx8fHx8fHx8fHx8fHx8fHx8fHx8fHx8fHx8fHx8fHx8TExMTExMTExMTExMTExMTExMTExMTERERERERERERERERERERERERERERERERERETExMTExMTExMTExMfHx8fHx8fHx8fHx8fHx8fHh4eHh4eHh4eHh4FBQUFBQUFBQUfHx8fHx8fHx8fHx8fHx8fHx8fHx8fHx8TExMTExMTExMTExMTExMTExMTExMTExMTExMTExERERERERERERERERERERERERERERERERERERERERMTExMTExMTExMTHx8fHx8fHx8fHx8fHx8eHh4eHh4FBQUFBQUFBQUFBQUFBR8fHx8fHx8fHx8fHx8fHx8TExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMRERERERERERERERERERERERERERERERERERERERERERERExMTExMTExMTExMTHx8fHx8fHx8fHx8eHh4eBQUFBQUFBQUFBQUFBQUFHx8fHx8fExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTExMTExMTExMTHx8fHx8fHx8eHgUFBQUFBQUFBQUFBQUFBQUTHx8TExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTExMTExMTExMTExMTEx8fHh4eBQUFBQUFBQUFBQUFBQUFBRMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTExMTExMTExMTExMTExMFBQUFBQUFBQUFBQUFBQUFExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTExMTExMTExMTExMTEwUFBQUFBQUFBQUFBQUFBQUTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMRERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTExMTExMTExMTBQUFBQUFBQUFBQUFBQUFBRMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERETExMTExMTExMTEwUFBQUFBQUFBQUFBQUFBQUFExMTExMTExMTExMTExMTExMTExMTExMTExMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERETExMTExMTBQUFBQUFBQUFBQUFBQUFBQUTExMTExMTExMTExMTExMTExMTExMTExMTEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREFBQUFBQUFBQUFBQUFBQUFBRMTExMTExMTExMTExMTExMTExMTEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREQUFBQUFBQUFBQUFBQUFBQUFExMTExMTExMTExMTExMTExMTEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREQUFBQUFBQUFBQUFBQUFBQUTExMTExMTExMTExMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERBQUFBQUFBQUFBQUFBQUFBRMTExMTExMTExMTExMTExEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREFBQUFBQUFBQUFBQUFBQUFExMTExMTExMTExMTExEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREFBQUFBQUFBQUFBQUFBQUTExMTExMTExMTExMREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREKEREREREREREREREREREREREREREREQUFBQUFBQUFBQUFBQUFBRMTExMTExMTExMTEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREQUFBQUFBQUFBQUFBQUFExMTExMTExMTExEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREFBQUFBQUFBQUFBQUTExMTExMTExMREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREFBQUFBQUFBRMTExMTExMTEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREQUFBQUFERETExMRERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERCgoRERERERERERERERERERERERERERERERERERERBQURERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTExMRERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERETExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERETExMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTExERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTExMRERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERETExMTExMRERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERMTExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERExMTERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERChEREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREKCgoREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREREKChERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERERER"
}

In [ ]:
import base64
import numpy as np
import matplotlib.pyplot as plt

pred_decoded_byte = base64.decodebytes(bytes(response["base64_prediction"], encoding="utf-8"))
pred_decoded = np.reshape(
    np.frombuffer(pred_decoded_byte, dtype=np.uint8), (96, 128)
)
plt.imshow(pred_decoded);